In [2]:
# import spark sql and functions
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.master('local[16]').getOrCreate()

22/11/16 22:56:23 WARN Utils: Your hostname, extrieve resolves to a loopback address: 127.0.1.1; using 192.168.0.25 instead (on interface wlo1)
22/11/16 22:56:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/16 22:56:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/16 22:56:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/16 22:56:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
# read products_table.csv
products = spark.read.csv('../Data/product_table.csv', header=True, inferSchema=True)

# schema
products.printSchema()

root
 |-- UPC: long (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- MANUFACTURER: string (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- SUB_CATEGORY: string (nullable = true)
 |-- PRODUCT_SIZE: string (nullable = true)



In [5]:
# store_table.csv
store = spark.read.csv('../Data/store_table.csv', header=True, inferSchema=True)

# schema
store.printSchema()

root
 |-- STORE_ID: integer (nullable = true)
 |-- STORE_NAME: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- MSA: integer (nullable = true)
 |-- SEGMENT: string (nullable = true)
 |-- PARKING: integer (nullable = true)
 |-- SIZE: integer (nullable = true)
 |-- AVG_WEEKLY_BASKETS: integer (nullable = true)
 |-- _c9: string (nullable = true)



In [6]:
# transactions_table.csv
transactions = spark.read.csv('../Data/transaction_table.csv', header=True, inferSchema=True)

In [7]:
# schema
transactions.printSchema()

root
 |-- WEEK_END_DATE: string (nullable = true)
 |-- STORE_NUM: integer (nullable = true)
 |-- UPC: long (nullable = true)
 |-- UNITS: integer (nullable = true)
 |-- VISITS: integer (nullable = true)
 |-- HHS: integer (nullable = true)
 |-- SPEND: double (nullable = true)
 |-- PRICE: double (nullable = true)
 |-- BASE_PRICE: double (nullable = true)
 |-- FEATURE: integer (nullable = true)
 |-- DISPLAY: integer (nullable = true)
 |-- TPR_ONLY: integer (nullable = true)



In [8]:
# Left join products and transactions
products_transactions = products.join(transactions, products.UPC == transactions.UPC, how='left')

# head
products_transactions.show(5)

+----------+--------------------+-------------+----------+------------+------------+-------------+---------+----------+-----+------+---+------+-----+----------+-------+-------+--------+
|       UPC|         DESCRIPTION| MANUFACTURER|  CATEGORY|SUB_CATEGORY|PRODUCT_SIZE|WEEK_END_DATE|STORE_NUM|       UPC|UNITS|VISITS|HHS| SPEND|PRICE|BASE_PRICE|FEATURE|DISPLAY|TPR_ONLY|
+----------+--------------------+-------------+----------+------------+------------+-------------+---------+----------+-----+------+---+------+-----+----------+-------+-------+--------+
|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|    14-Jan-09|      367|1111009477|   13|    13| 13| 18.07| 1.39|      1.57|      0|      0|       1|
|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|    14-Jan-09|      387|1111009477|  111|    98| 94|154.29| 1.39|      1.39|      0|      0|       0|
|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS

In [10]:
# schema
products_transactions.printSchema()

root
 |-- UPC: long (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- MANUFACTURER: string (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- SUB_CATEGORY: string (nullable = true)
 |-- PRODUCT_SIZE: string (nullable = true)
 |-- WEEK_END_DATE: string (nullable = true)
 |-- STORE_NUM: integer (nullable = true)
 |-- UPC: long (nullable = true)
 |-- UNITS: integer (nullable = true)
 |-- VISITS: integer (nullable = true)
 |-- HHS: integer (nullable = true)
 |-- SPEND: double (nullable = true)
 |-- PRICE: double (nullable = true)
 |-- BASE_PRICE: double (nullable = true)
 |-- FEATURE: integer (nullable = true)
 |-- DISPLAY: integer (nullable = true)
 |-- TPR_ONLY: integer (nullable = true)



In [12]:
# Left join products_transactions and store, store_id == store_num
products_transactions_store = products_transactions.join(store, products_transactions.STORE_NUM == store.STORE_ID, how='left')

# head
products_transactions_store.show(5)

22/11/16 23:07:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/11/16 23:07:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: STORE_ID, STORE_NAME, CITY, STATE, MSA, SEGMENT, PARKING, SIZE, AVG_WEEKLY_BASKETS, 
 Schema: STORE_ID, STORE_NAME, CITY, STATE, MSA, SEGMENT, PARKING, SIZE, AVG_WEEKLY_BASKETS, _c9
Expected: _c9 but found: 
CSV file: file:///home/nick/Documents/GitHub/BigDataProject/Data/store_table.csv


+----------+--------------------+-------------+----------+------------+------------+-------------+---------+----------+-----+------+---+------+-----+----------+-------+-------+--------+--------+--------------+---------+-----+-----+----------+-------+-----+------------------+----+
|       UPC|         DESCRIPTION| MANUFACTURER|  CATEGORY|SUB_CATEGORY|PRODUCT_SIZE|WEEK_END_DATE|STORE_NUM|       UPC|UNITS|VISITS|HHS| SPEND|PRICE|BASE_PRICE|FEATURE|DISPLAY|TPR_ONLY|STORE_ID|    STORE_NAME|     CITY|STATE|  MSA|   SEGMENT|PARKING| SIZE|AVG_WEEKLY_BASKETS| _c9|
+----------+--------------------+-------------+----------+------------+------------+-------------+---------+----------+-----+------+---+------+-----+----------+-------+-------+--------+--------+--------------+---------+-----+-----+----------+-------+-----+------------------+----+
|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|    14-Jan-09|      367|1111009477|   13|    13| 13| 18.07| 1.39|      1.

In [14]:
# schema
products_transactions_store.printSchema()

root
 |-- UPC: long (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- MANUFACTURER: string (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- SUB_CATEGORY: string (nullable = true)
 |-- PRODUCT_SIZE: string (nullable = true)
 |-- WEEK_END_DATE: string (nullable = true)
 |-- STORE_NUM: integer (nullable = true)
 |-- UPC: long (nullable = true)
 |-- UNITS: integer (nullable = true)
 |-- VISITS: integer (nullable = true)
 |-- HHS: integer (nullable = true)
 |-- SPEND: double (nullable = true)
 |-- PRICE: double (nullable = true)
 |-- BASE_PRICE: double (nullable = true)
 |-- FEATURE: integer (nullable = true)
 |-- DISPLAY: integer (nullable = true)
 |-- TPR_ONLY: integer (nullable = true)
 |-- STORE_ID: integer (nullable = true)
 |-- STORE_NAME: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- MSA: integer (nullable = true)
 |-- SEGMENT: string (nullable = true)
 |-- PARKING: integer (nullable = true)
 |-- S

In [15]:
# Drop similar columns
products_transactions_store = products_transactions_store.drop('UPC', 'STORE_NUM', 'STORE_ID')

# head
products_transactions_store.show(5)

22/11/16 23:09:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: STORE_ID, STORE_NAME, CITY, STATE, MSA, SEGMENT, PARKING, SIZE, AVG_WEEKLY_BASKETS, 
 Schema: STORE_ID, STORE_NAME, CITY, STATE, MSA, SEGMENT, PARKING, SIZE, AVG_WEEKLY_BASKETS, _c9
Expected: _c9 but found: 
CSV file: file:///home/nick/Documents/GitHub/BigDataProject/Data/store_table.csv


+--------------------+-------------+----------+------------+------------+-------------+-----+------+---+------+-----+----------+-------+-------+--------+--------------+---------+-----+-----+----------+-------+-----+------------------+----+
|         DESCRIPTION| MANUFACTURER|  CATEGORY|SUB_CATEGORY|PRODUCT_SIZE|WEEK_END_DATE|UNITS|VISITS|HHS| SPEND|PRICE|BASE_PRICE|FEATURE|DISPLAY|TPR_ONLY|    STORE_NAME|     CITY|STATE|  MSA|   SEGMENT|PARKING| SIZE|AVG_WEEKLY_BASKETS| _c9|
+--------------------+-------------+----------+------------+------------+-------------+-----+------+---+------+-----+----------+-------+-------+--------+--------------+---------+-----+-----+----------+-------+-----+------------------+----+
|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|    14-Jan-09|   13|    13| 13| 18.07| 1.39|      1.57|      0|      0|       1|15TH & MADISON|COVINGTON|   KY|17140|     VALUE|    196|24721|             12707|null|
|PL MINI TWIST PRE...|PRIVATE LABEL|BAG 

In [17]:
# shape of the data
print((products_transactions_store.count(), len(products_transactions_store.columns)))

(538662, 24)


In [18]:
# drop any duplicates
products_transactions_store = products_transactions_store.dropDuplicates()

# shape of the data
print((products_transactions_store.count(), len(products_transactions_store.columns)))

22/11/16 23:10:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: STORE_ID, STORE_NAME, CITY, STATE, MSA, SEGMENT, PARKING, SIZE, AVG_WEEKLY_BASKETS, 
 Schema: STORE_ID, STORE_NAME, CITY, STATE, MSA, SEGMENT, PARKING, SIZE, AVG_WEEKLY_BASKETS, _c9
Expected: _c9 but found: 
CSV file: file:///home/nick/Documents/GitHub/BigDataProject/Data/store_table.csv


(538647, 24)


In [22]:
# to pandas
products_transactions_store = products_transactions_store.toPandas()

# save to csv
products_transactions_store.to_csv('../Data/products_transactions_store.csv', index=False)

22/11/16 23:17:05 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks 
java.io.IOException: Connecting to /192.168.0.25:38989 failed in the last 4750 ms, fail this connection directly
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:214)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:126)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:154)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.start(RetryingBlockTransferor.java:133)
	at org.apache.spark.network.netty.NettyBlockTransferService.fetchBlocks(NettyBlockTransferService.scala:146)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:102)
	at org.apache.spark.storage.BlockManager.fetchRemoteManagedBuffer(BlockManager.scala:1154)
	at org.apache.spark.storage.BlockMan

Py4JJavaError: An error occurred while calling o85.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 70.0 failed 1 times, most recent failure: Lost task 5.0 in stage 70.0 (TID 190) (192.168.0.25 executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(